In [0]:
%pip install databricks-sql-connector pandas openpyxl numpy

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## add your databricks warehouse credentials here

In [0]:
import os 
    
os.environ['DATABRICKS_HOST'] = '*'
os.environ['DATABRICKS_HTTP_PATH'] = '*'
os.environ['DATABRICKS_TOKEN'] = '*'
os.environ['DATABRICKS_CATALOG']='*'
os.environ['DATABRICKS_SCHEMA']='*'

2025-12-16 21:44:39,189 - INFO - Received command c on object id p0


In [0]:
# The code is mostly correct, but replace !pip with %pip for Databricks compatibility.
from datetime import datetime
from databricks import sql
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
import logging
import os
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TaxiReportGenerator:
    def __init__(self, server_hostname, http_path, access_token, catalog, schema):
        """
        Initialize Databricks connection
        
        Args:
            server_hostname: Databricks workspace URL (e.g., 'adb-xxxxxxx.azuredatabricks.net')
            http_path: SQL warehouse HTTP path (e.g., '/sql/1.0/warehouses/xxxxxxx')
            access_token: Personal access token or service principal token
            catalog: Catalog name (e.g., 'main')
            schema: Schema name (e.g., 'taxi_warehouse')
        """
        self.server_hostname = server_hostname
        self.http_path = http_path
        self.access_token = access_token
        self.catalog = catalog
        self.schema = schema
        self.conn = None
        self.output_file = f'Taxi_Report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
        
    def connect(self):
        """Establish Databricks connection"""
        try:
            self.conn = sql.connect(
                server_hostname=self.server_hostname,
                http_path=self.http_path,
                access_token=self.access_token
            )
            logger.info("✓ Databricks connection established")
            return True
        except Exception as e:
            logger.error(f"✗ Connection failed: {e}")
            return False
    
    def query(self, sql_query):
        """Execute SQL query and return DataFrame"""
        try:
            cursor = self.conn.cursor()
            cursor.execute(sql_query)
            result = cursor.fetchall()
            
            # Get column names
            columns = [desc[0] for desc in cursor.description]
            
            # Convert to DataFrame
            df = pd.DataFrame(result, columns=columns)
            cursor.close()
            return df
        except Exception as e:
            logger.error(f"✗ Query failed: {e}")
            return pd.DataFrame()
    
    def get_executive_kpis(self):
        """Get executive overview KPIs"""
        logger.info("📊 Fetching Executive KPIs...")
        
        sql_query = f"""
        SELECT 
            COUNT(DISTINCT trip_id) as total_trips,
            ROUND(SUM(fare_amount), 2) as total_revenue,
            SUM(passenger_count) as total_passengers,
            ROUND(AVG(trip_duration_minutes), 2) as avg_duration,
            ROUND(AVG(trip_distance_miles), 2) as avg_distance,
            ROUND(AVG(fare_amount), 2) as avg_fare_per_trip,
            ROUND(SUM(fare_amount) / COUNT(DISTINCT trip_id), 2) as revenue_per_trip
        FROM {self.catalog}.{self.schema}.fact_trips
        """
        
        return self.query(sql_query)
    
    def get_executive_zones(self):
        """Get top zones by trip count and revenue"""
        logger.info("📊 Fetching Executive Zone Data...")
        
        sql_query = f"""
        SELECT 
            z.zone_name,
            z.borough,
            COUNT(DISTINCT ft.trip_id) as trips,
            ROUND(SUM(ft.fare_amount), 2) as revenue,
            ROUND(COUNT(DISTINCT ft.trip_id) * 100.0 / 
                  (SELECT COUNT(*) FROM {self.catalog}.{self.schema}.fact_trips), 2) as pct_total
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_location z ON ft.pickup_location_key = z.location_key
        GROUP BY z.zone_name, z.borough
        ORDER BY trips DESC
        LIMIT 10
        """
        
        return self.query(sql_query)
    
    def get_demand_summary(self):
        """Get demand analysis summary"""
        logger.info("📊 Fetching Demand Summary...")
        
        sql_query = f"""
        SELECT 
            COUNT(DISTINCT trip_id) as total_trips,
            SUM(passenger_count) as total_passengers,
            ROUND(AVG(CAST(passenger_count AS FLOAT)), 2) as avg_passengers_per_trip
        FROM {self.catalog}.{self.schema}.fact_trips
        """
        
        return self.query(sql_query)
    
    def get_demand_heatmap(self):
        """Get demand heatmap by day of week and hour"""
        logger.info("📊 Fetching Demand Heatmap...")
        
        sql_query = f"""
        SELECT 
            dd.day_name,
            HOUR(from_unixtime(cast(dd.date_key as bigint) / 1000)) as hour,
            COUNT(DISTINCT ft.trip_id) as trip_count
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_date dd ON ft.pickup_date_key = dd.date_key
        GROUP BY dd.day_name, HOUR(from_unixtime(cast(dd.date_key as bigint) / 1000))
        ORDER BY 
            CASE dd.day_name 
                WHEN 'Monday' THEN 1
                WHEN 'Tuesday' THEN 2
                WHEN 'Wednesday' THEN 3
                WHEN 'Thursday' THEN 4
                WHEN 'Friday' THEN 5
                WHEN 'Saturday' THEN 6
                WHEN 'Sunday' THEN 7
            END, hour
        """
        
        return self.query(sql_query)
    
    def get_revenue_by_zone(self):
        """Get revenue analysis by zone"""
        logger.info("💰 Fetching Revenue by Zone...")
        
        sql_query = f"""
        SELECT 
            z.zone_name,
            COUNT(DISTINCT ft.trip_id) as trip_count,
            ROUND(SUM(ft.fare_amount), 2) as total_revenue,
            ROUND(AVG(ft.fare_amount), 2) as avg_fare,
            ROUND(SUM(ft.fare_amount) / SUM(ft.trip_distance_miles), 2) as revenue_per_km
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_location z ON ft.pickup_location_key = z.location_key
        GROUP BY z.zone_name
        ORDER BY total_revenue DESC
        LIMIT 10
        """
        
        return self.query(sql_query)
    
    def get_revenue_by_vendor(self):
        """Get revenue analysis by vendor"""
        logger.info("💰 Fetching Revenue by Vendor...")
        
        sql_query = f"""
        SELECT 
            v.vendor_name,
            COUNT(DISTINCT ft.trip_id) as trips,
            ROUND(SUM(ft.fare_amount), 2) as total_revenue,
            ROUND(AVG(ft.fare_amount), 2) as avg_fare,
            ROUND(SUM(ft.fare_amount) / COUNT(DISTINCT ft.trip_id), 2) as revenue_per_trip
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_vendor v ON ft.vendor_key = v.vendor_key
        GROUP BY v.vendor_name
        ORDER BY total_revenue DESC
        """
        
        return self.query(sql_query)
    
    def get_efficiency_summary(self):
        """Get trip efficiency metrics"""
        logger.info("⚡ Fetching Efficiency Metrics...")
        
        sql_query = f"""
        SELECT 
            ROUND(AVG(trip_duration_minutes), 2) as avg_duration_min,
            ROUND(AVG(trip_distance_miles), 2) as avg_distance_miles,
            ROUND(AVG(trip_distance_miles) / NULLIF(AVG(trip_duration_minutes), 0) * 60, 2) as avg_speed_kmh
        FROM {self.catalog}.{self.schema}.fact_trips
        """
        
        return self.query(sql_query)
    
    def get_efficiency_by_zone(self):
        """Get efficiency metrics by zone"""
        logger.info("⚡ Fetching Efficiency by Zone...")
        
        sql_query = f"""
        SELECT 
            z.zone_name,
            ROUND(AVG(ft.trip_duration_minutes), 2) as avg_duration,
            ROUND(AVG(ft.trip_distance_miles), 2) as avg_distance,
            ROUND(AVG(ft.trip_distance_miles) / NULLIF(AVG(ft.trip_duration_minutes), 0) * 60, 2) as avg_speed
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_location z ON ft.pickup_location_key = z.location_key
        GROUP BY z.zone_name
        ORDER BY avg_speed DESC
        LIMIT 10
        """
        
        return self.query(sql_query)
    
    def get_spatial_flow(self):
        """Get top routes (pickup → dropoff)"""
        logger.info("🗺️  Fetching Spatial Flow Data...")
        
        sql_query = f"""
        SELECT 
            zp.zone_name as pickup_zone,
            zd.zone_name as dropoff_zone,
            COUNT(DISTINCT ft.trip_id) as trip_count,
            ROUND(AVG(ft.trip_duration_minutes), 2) as avg_duration,
            ROUND(AVG(ft.trip_distance_miles), 2) as avg_distance,
            ROUND(SUM(ft.fare_amount), 2) as total_revenue
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_location zp ON ft.pickup_location_key = zp.location_key
        JOIN {self.catalog}.{self.schema}.dim_location zd ON ft.dropoff_location_key = zd.location_key
        GROUP BY zp.zone_name, zd.zone_name
        ORDER BY trip_count DESC
        LIMIT 10
        """
        
        return self.query(sql_query)
    
    def get_passenger_distribution(self):
        """Get passenger count distribution"""
        logger.info("👥 Fetching Passenger Distribution...")
        
        sql_query = f"""
        SELECT 
            passenger_count,
            COUNT(DISTINCT trip_id) as trip_count,
            ROUND(COUNT(DISTINCT trip_id) * 100.0 / 
                  (SELECT COUNT(*) FROM {self.catalog}.{self.schema}.fact_trips), 2) as pct_total,
            ROUND(AVG(fare_amount), 2) as avg_revenue
        FROM {self.catalog}.{self.schema}.fact_trips
        GROUP BY passenger_count
        ORDER BY passenger_count
        """
        
        return self.query(sql_query)
    
    def get_payment_analysis(self):
        """Get payment type analysis"""
        logger.info("💳 Fetching Payment Analysis...")
        
        sql_query = f"""
        SELECT 
            p.payment_name,
            COUNT(DISTINCT ft.trip_id) as trips,
            ROUND(SUM(ft.fare_amount), 2) as total_revenue,
            ROUND(AVG(ft.fare_amount), 2) as avg_fare,
            p.is_electronic
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_payment_type p ON ft.payment_type_key = p.payment_type_key
        GROUP BY p.payment_name, p.is_electronic
        ORDER BY total_revenue DESC
        """
        
        return self.query(sql_query)
    
    def get_payment_by_rate(self):
        """Get analysis by rate code"""
        logger.info("💳 Fetching Rate Code Analysis...")
        
        sql_query = f"""
        SELECT 
            r.rate_code_desc,
            COUNT(DISTINCT ft.trip_id) as trips,
            ROUND(SUM(ft.fare_amount), 2) as total_revenue,
            ROUND(AVG(ft.fare_amount), 2) as avg_fare
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_rate_code r ON ft.rate_code_key = r.rate_code_key
        GROUP BY r.rate_code_desc
        ORDER BY total_revenue DESC
        """
        
        return self.query(sql_query)
    
    def get_vendor_performance(self):
        """Get comprehensive vendor performance"""
        logger.info("🚕 Fetching Vendor Performance...")
        
        sql_query = f"""
        SELECT 
            v.vendor_name,
            COUNT(DISTINCT ft.trip_id) as total_trips,
            ROUND(SUM(ft.fare_amount), 2) as total_revenue,
            ROUND(AVG(ft.trip_duration_minutes), 2) as avg_duration,
            ROUND(AVG(ft.trip_distance_miles), 2) as avg_distance,
            ROUND(SUM(ft.fare_amount) / COUNT(DISTINCT ft.trip_id), 2) as revenue_per_trip,
            SUM(ft.passenger_count) as total_passengers,
            ROUND(AVG(CAST(ft.passenger_count AS FLOAT)), 2) as avg_passengers
        FROM {self.catalog}.{self.schema}.fact_trips ft
        JOIN {self.catalog}.{self.schema}.dim_vendor v ON ft.vendor_key = v.vendor_key
        GROUP BY v.vendor_name
        ORDER BY total_revenue DESC
        """
        
        return self.query(sql_query)
    
    def format_excel_header(self, ws, row):
        """Format header row with styling"""
        header_fill = PatternFill(start_color="667EEA", end_color="667EEA", fill_type="solid")
        header_font = Font(color="FFFFFF", bold=True, size=11)
        border = Border(left=Side(style='thin'), right=Side(style='thin'),
                       top=Side(style='thin'), bottom=Side(style='thin'))
        
        for cell in ws[row]:
            if cell.value:
                cell.fill = header_fill
                cell.font = header_font
                cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
                cell.border = border
    
    def format_data_rows(self, ws, start_row, end_row):
        """Format data rows"""
        border = Border(left=Side(style='thin'), right=Side(style='thin'),
                       top=Side(style='thin'), bottom=Side(style='thin'))
        
        for row in ws.iter_rows(min_row=start_row, max_row=end_row):
            for cell in row:
                cell.border = border
                cell.alignment = Alignment(horizontal='left', vertical='center')
                if isinstance(cell.value, (int, float)):
                    cell.alignment = Alignment(horizontal='right', vertical='center')
    
    def autofit_columns(self, ws):
        """Auto-fit column widths"""
        for column in ws.columns:
            max_length = 0
            column_letter = get_column_letter(column[0].column)
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 50)
            ws.column_dimensions[column_letter].width = adjusted_width
    
    def create_report(self):
        """Generate complete Excel report"""
        if not self.connect():
            return False
        
        try:
            with pd.ExcelWriter(self.output_file, engine='openpyxl') as writer:
                # Executive Overview
                logger.info("\n📄 Creating Executive Overview Sheet...")
                kpis = self.get_executive_kpis()
                zones = self.get_executive_zones()
                
                kpis.to_excel(writer, sheet_name='Executive KPIs', index=False, startrow=0)
                zones.to_excel(writer, sheet_name='Executive KPIs', index=False, startrow=len(kpis) + 3)
                
                # Demand Analysis
                logger.info("📄 Creating Demand Analysis Sheet...")
                demand_summary = self.get_demand_summary()
                demand_heatmap = self.get_demand_heatmap()
                
                demand_summary.to_excel(writer, sheet_name='Demand', index=False, startrow=0)
                demand_heatmap.to_excel(writer, sheet_name='Demand', index=False, startrow=len(demand_summary) + 3)
                
                # Revenue Analysis
                logger.info("📄 Creating Revenue Analysis Sheet...")
                revenue_zone = self.get_revenue_by_zone()
                revenue_vendor = self.get_revenue_by_vendor()
                
                revenue_zone.to_excel(writer, sheet_name='Revenue', index=False, startrow=0)
                revenue_vendor.to_excel(writer, sheet_name='Revenue', index=False, startrow=len(revenue_zone) + 3)
                
                # Trip Efficiency
                logger.info("📄 Creating Efficiency Sheet...")
                efficiency_summary = self.get_efficiency_summary()
                efficiency_zone = self.get_efficiency_by_zone()
                
                efficiency_summary.to_excel(writer, sheet_name='Efficiency', index=False, startrow=0)
                efficiency_zone.to_excel(writer, sheet_name='Efficiency', index=False, startrow=3)
                
                # Spatial Flow
                logger.info("📄 Creating Spatial Flow Sheet...")
                spatial = self.get_spatial_flow()
                spatial.to_excel(writer, sheet_name='Spatial Flow', index=False)
                
                # Passenger Behavior
                logger.info("📄 Creating Passenger Behavior Sheet...")
                passengers = self.get_passenger_distribution()
                passengers.to_excel(writer, sheet_name='Passenger Behavior', index=False)
                
                # Payment & Pricing
                logger.info("📄 Creating Payment & Pricing Sheet...")
                payment = self.get_payment_analysis()
                payment_rate = self.get_payment_by_rate()
                
                payment.to_excel(writer, sheet_name='Payment', index=False, startrow=0)
                payment_rate.to_excel(writer, sheet_name='Payment', index=False, startrow=len(payment) + 3)
                
                # Vendor Performance
                logger.info("📄 Creating Vendor Performance Sheet...")
                vendor = self.get_vendor_performance()
                vendor.to_excel(writer, sheet_name='Vendor Performance', index=False)
            
            # Apply formatting
            logger.info("\n🎨 Applying formatting...")
            self.apply_formatting()
            
            logger.info(f"\n✓ Report generated successfully: {self.output_file}")
            return True
            
        except Exception as e:
            logger.error(f"\n✗ Report generation failed: {e}")
            return False
        finally:
            if self.conn:
                self.conn.close()
    
    def apply_formatting(self):
        """Apply formatting to all sheets"""
        wb = load_workbook(self.output_file)
        
        for sheet_name in wb.sheetnames:
            ws = wb[sheet_name]
            
            # Format header (first row)
            self.format_excel_header(ws, 1)
            
            # Format data rows
            if ws.max_row > 1:
                self.format_data_rows(ws, 2, ws.max_row)
            
            # Auto-fit columns
            self.autofit_columns(ws)
        
        wb.save(self.output_file)


def main():
    """Main execution"""
    print("=" * 60)
    print("🚕 Taxi Data Warehouse - Databricks Report Generator")
    print("=" * 60)
    
    # Databricks connection parameters
    # You can also load these from environment variables or config file
    SERVER_HOSTNAME = os.getenv('DATABRICKS_HOST', 'your-workspace.azuredatabricks.net')
    HTTP_PATH = os.getenv('DATABRICKS_HTTP_PATH', '/sql/1.0/warehouses/your-warehouse-id')
    ACCESS_TOKEN = os.getenv('DATABRICKS_TOKEN', 'your-access-token')
    CATALOG = os.getenv('DATABRICKS_CATALOG', 'main')
    SCHEMA = os.getenv('DATABRICKS_SCHEMA', 'taxi_warehouse')
    
    # Create generator and generate report
    generator = TaxiReportGenerator(
        server_hostname=SERVER_HOSTNAME,
        http_path=HTTP_PATH,
        access_token=ACCESS_TOKEN,
        catalog=CATALOG,
        schema=SCHEMA
    )
    
    if generator.create_report():
        print("\n" + "=" * 60)
        print(f"✓ Report saved as: {generator.output_file}")
        print("=" * 60)
    else:
        print("\n" + "=" * 60)
        print("✗ Failed to generate report")
        print("=" * 60)


if __name__ == "__main__":
    main()


2025-12-16 21:44:41,382 - INFO - Received status code 200 for POST request
2025-12-16 21:44:41,383 - INFO - HTTP Response with status code 200, message: OK
2025-12-16 21:44:41,384 - INFO - Successfully opened session 01f0dac8-6d7a-1cec-a9ca-5c23bbe5f4e8
2025-12-16 21:44:41,385 - INFO - ✓ Databricks connection established


🚕 Taxi Data Warehouse - Databricks Report Generator


2025-12-16 21:44:41,446 - INFO - 
📄 Creating Executive Overview Sheet...
2025-12-16 21:44:41,447 - INFO - 📊 Fetching Executive KPIs...
2025-12-16 21:44:46,744 - INFO - Received status code 200 for POST request
2025-12-16 21:44:46,744 - INFO - HTTP Response with status code 200, message: OK
2025-12-16 21:44:51,146 - INFO - Received status code 200 for POST request
2025-12-16 21:44:51,147 - INFO - HTTP Response with status code 200, message: OK
2025-12-16 21:44:51,232 - INFO - Received status code 200 for POST request
2025-12-16 21:44:51,233 - INFO - HTTP Response with status code 200, message: OK
2025-12-16 21:44:51,386 - INFO - Received status code 200 for POST request
2025-12-16 21:44:51,387 - INFO - HTTP Response with status code 200, message: OK
2025-12-16 21:44:51,501 - INFO - Received status code 200 for POST request
2025-12-16 21:44:51,502 - INFO - HTTP Response with status code 200, message: OK
2025-12-16 21:44:51,503 - INFO - 📊 Fetching Executive Zone Data...
2025-12-16 21:44:5


✓ Report saved as: Taxi_Report_20251216_214441.xlsx
